In [137]:
import pandas as pd

In [138]:
import matplotlib.pyplot as plt

In [139]:
df= pd.read_csv('all_indi/XMLProduct_annotations.csv')

In [140]:
df = df[~df.Annotation.isin(['DISEASE','CANCER','SYNDROME'])]

In [141]:
df.Annotation.value_counts()

HYPERTENSION                       7688
STROKE                             4254
BIPOLAR I DISORDER                 3984
MYOCARDIAL INFARCTION              3696
SCHIZOPHRENIA                      3476
ANXIETY                            2493
CORONARY HEART DISEASE             2338
MAJOR DEPRESSIVE DISORDER          2119
PANIC DISORDER                     2119
CHLAMYDIA                          2014
PNEUMONIA                          1846
HYPERCHOLESTEROLEMIA               1652
GERD                               1650
FAMILIAL HYPERCHOLESTEROLEMIA      1618
DYSLIPIDEMIA                       1440
GENERALIZED ANXIETY DISORDER       1396
KIDNEY DISEASE                     1353
DUODENAL ULCER                     1208
ESOPHAGITIS                        1107
GASTRIC ULCER                      1097
MIGRAINE                           1048
BREAST CANCER                      1009
EPILEPSY                            991
INFLUENZA                           953
BIPOLAR DISORDER                    901


In [142]:
df.head()

,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName,Drug_Brand_Name
1,1bd850c4-a638-4143-b6be-0c50521a0449.xml,1036,1058,SYN,CARDIOPULMONARY DISEASE,"Ribavirin for Inhalation Solution, USP is indi...",http://purl.obolibrary.org/obo/DOID_8515,DB00811,Ribavirin,Ribavirin
2,1bd850c4-a638-4143-b6be-0c50521a0449.xml,1908,1926,PREF,BACTERIAL PNEUMONIA,"Ribavirin for Inhalation Solution, USP is indi...",http://purl.obolibrary.org/obo/DOID_874,DB00811,Ribavirin,Ribavirin
4,0028a8d7-5329-4faf-a7e0-ba3abb26b9b4.xml,1036,1058,SYN,CARDIOPULMONARY DISEASE,"Ribavirin for inhalation solution, USP is indi...",http://purl.obolibrary.org/obo/DOID_8515,DB00811,RIBAVIRIN,RIBAVIRIN
5,0028a8d7-5329-4faf-a7e0-ba3abb26b9b4.xml,1891,1909,PREF,BACTERIAL PNEUMONIA,"Ribavirin for inhalation solution, USP is indi...",http://purl.obolibrary.org/obo/DOID_874,DB00811,RIBAVIRIN,RIBAVIRIN
6,3fa42a10-6874-4b83-945f-10d7668d2138.xml,67,85,SYN,CHRONIC HEPATITIS C,Ribavirin is a nucleoside analogue indicated f...,http://purl.obolibrary.org/obo/DOID_1883,DB00811,RIBAVIRIN,Ribavirin


In [143]:
len(df)

122190

In [144]:
db_atc = pd.read_csv('data/input/drugbank_atc_codes.csv') 

In [145]:
len(db_atc)

4963

In [146]:
db_atc = db_atc.drop_duplicates(subset=['dbid'],keep='first')

In [147]:
len(db_atc)

3145

In [148]:
db_atc = db_atc.rename(columns={'dbid':'DB_ID'})

In [149]:
df = df.merge(db_atc,on=['DB_ID'])

In [150]:
len(df)

119067

In [151]:
df.head()

,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName,Drug_Brand_Name,atc_label
0,1bd850c4-a638-4143-b6be-0c50521a0449.xml,1036,1058,SYN,CARDIOPULMONARY DISEASE,"Ribavirin for Inhalation Solution, USP is indi...",http://purl.obolibrary.org/obo/DOID_8515,DB00811,Ribavirin,Ribavirin,J05AP01
1,1bd850c4-a638-4143-b6be-0c50521a0449.xml,1908,1926,PREF,BACTERIAL PNEUMONIA,"Ribavirin for Inhalation Solution, USP is indi...",http://purl.obolibrary.org/obo/DOID_874,DB00811,Ribavirin,Ribavirin,J05AP01
2,0028a8d7-5329-4faf-a7e0-ba3abb26b9b4.xml,1036,1058,SYN,CARDIOPULMONARY DISEASE,"Ribavirin for inhalation solution, USP is indi...",http://purl.obolibrary.org/obo/DOID_8515,DB00811,RIBAVIRIN,RIBAVIRIN,J05AP01
3,0028a8d7-5329-4faf-a7e0-ba3abb26b9b4.xml,1891,1909,PREF,BACTERIAL PNEUMONIA,"Ribavirin for inhalation solution, USP is indi...",http://purl.obolibrary.org/obo/DOID_874,DB00811,RIBAVIRIN,RIBAVIRIN,J05AP01
4,3fa42a10-6874-4b83-945f-10d7668d2138.xml,67,85,SYN,CHRONIC HEPATITIS C,Ribavirin is a nucleoside analogue indicated f...,http://purl.obolibrary.org/obo/DOID_1883,DB00811,RIBAVIRIN,Ribavirin,J05AP01


In [152]:
df['ATC_LEVEL_0']= df['atc_label'].str[0]

In [153]:
groups = df.groupby('ATC_LEVEL_0')

In [154]:
selected = pd.DataFrame()
for name, group in groups:
    print(name)
    if len(group) > 50:
        selected= selected.append(group.sample(n=50))
    else:
        selected= selected.append(group)

A
B
C
D
G
H
J
L
M
N
R
S
V


In [155]:
len(selected)

650

In [157]:
def getDiseaseTerm(row):
   #print (row)
    from_ = row['From']
    to_ = row['To']
    return row['Context'][from_-1:to_]

selected['DieaseName']= selected.apply(getDiseaseTerm, axis='columns')

In [158]:
counter = []
for index, row in selected.iterrows():
    test = row['Context']
    #Counts the number of entries at row x and adds it to the counter list
    counter.append(len(test.split()))

#df.Text.str.split('\n', expand=True).stack()
#The word count list is now appended to the context file
selected['WordCount'] = counter

In [159]:
selected =selected[selected.WordCount > 200]

In [160]:
selected.head()

,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName,Drug_Brand_Name,atc_label,ATC_LEVEL_0,DieaseName,WordCount
58204,4c9c1111-c13e-42fc-9c1d-5f9c54c86515.xml,74,87,PREF,DUODENAL ULCER,Omeprazole is a proton pump inhibitor indicate...,http://purl.obolibrary.org/obo/DOID_1724,DB00338,OMEPRAZOLE,Omeprazole,A02BC01,A,duodenal ulcer,397
96543,899a4340-9049-29a3-e053-2995a90a8e7f.xml,1187,1195,PREF,PSORIASIS,Prednisone tablets are indicated in the follow...,http://purl.obolibrary.org/obo/DOID_8893,DB00635,PREDNISONE,Prednisone,A07EA03,A,psoriasis,326
95881,280c9fe8-339a-177a-e054-00144ff88e88.xml,1155,1164,PREF,DERMATITIS,"Prednisone Tablets, USP are indicated in the f...",http://purl.obolibrary.org/obo/DOID_2723,DB00635,PREDNISONE,Prednisone,A07EA03,A,dermatitis,328
95179,5092299c-3782-4ee5-b3e7-4e80bf378e3b.xml,1821,1834,PREF,OPTIC NEURITIS,Prednisone tablets and solutions are indicated...,http://purl.obolibrary.org/obo/DOID_1210,DB00635,PREDNISONE,PredniSONE,A07EA03,A,optic neuritis,328
91987,bd6475fb-0a6a-44af-a99c-6c40766adf5e.xml,1308,1328,PREF,SEBORRHEIC DERMATITIS,"Prednisone tablets, USP are indicated in the f...",http://purl.obolibrary.org/obo/DOID_8741,DB00635,PREDNISONE,Prednisone,A07EA03,A,seborrheic dermatitis,351


In [161]:
len(selected)

494